# Scrape HTML Page to get the CANADA postal codes data from WIKIPEDIA and get the latitudes and longitudes for each neighborhood and explore Toronto Neighborhood

## Introduction

In this assignment, 
1. we will build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe
2. Get the latitude and the longitude coordinates of each neighborhood.
3. Explore, analyze and cluster the neighborhoods in Toronto

### Download all the dependencies that we will need

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
# import k-means from clustering stage
from sklearn.cluster import KMeans
# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

Make the request to the wikipedia page and create a beautiful soup object using the response

In [2]:
response = requests.get("http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(response.text)

### Below steps are performed
1. Find the table element in the Beautiful soup object, the first table element contains the details regarding the Postal Code, Borough & Neighborhood
2. Get the rows of the table and process each row based on the conditions provided in the assignment
3. Create a list which contains Postal Code, Borough & Neighborhood

In [3]:
tables = soup.findChildren('table')
my_table=tables[0]
rows = my_table.findChildren(['tr'])
datanew=[]
for row in rows:
    cells = row.findChildren('td')
    if cells:
        listnew=[]
        if(cells[1].string != "Not assigned"):
            listnew.append(cells[0].string)
            listnew.append(cells[1].string)
            anchor = cells[2].find_all('a')
            if(anchor):
                listnew.append(anchor[0].string)
            else:
                listnew.append(cells[1].string)
        if listnew:
            datanew.append(listnew)

Create a dataframe from the list

In [4]:
df = pd.DataFrame(datanew,columns = ['PostalCode', 'Borough', 'Neighborhood'])

Merge the rows where we have more than 1 neighborhood for the same postal code

In [5]:
df=df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: ','.join(x.astype(str))).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Scarborough,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Scarborough


In [6]:
df.shape

(103, 3)

Faced issues with geocoder package, so using the csv file to populate latitude and longitude

In [8]:
lat_log = pd.read_csv("http://cocl.us/Geospatial_data")
lat_log.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Rename Postal Code in lat_log data frame to PostalCode to match with the name in df data frame

In [9]:
lat_log=lat_log.rename(index=str, columns={"Postal Code": "PostalCode"})
lat_log.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the two data frames based on postal code to get the latitude and longitude values in neighborhoods data frame along with postal code, borough & neighborhood

In [10]:
neighborhoods = pd.merge(df, lat_log, on='PostalCode', how='inner')

In [11]:
neighborhoods.shape

(103, 5)

In [12]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>toronto_explorer</em>, as shown below.

In [13]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Let's simplify the above map and segment and cluster only the neighborhoods in boroughs that contain the word Toronto. So let's slice the original dataframe and create a new dataframe for this data

In [15]:
toronto_name_data = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_name_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"East Toronto,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"East Toronto,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,East Toronto,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


As we did with all of Toronto city, let's visualize only the neighborhoods whose borough contains Toronto in it's name

In [16]:
# create map of Neighborhoods whose borough contains Toronto in it's name using latitude and longitude values
map_toronto_name = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_name_data['Latitude'], toronto_name_data['Longitude'], toronto_name_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_name)  
    
map_toronto_name

Next, we will utilize the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [17]:
CLIENT_ID = 'HBZIXM2TB2MTGXPDQJIFGMW4AXJOKOE4WDKV11AROWDLEBDH' # your Foursquare ID
CLIENT_SECRET = 'QXPGNTYIPIIQZN1MYWXNKCAXWFPLTYBGI3YRJ1OYBYEGAFDZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HBZIXM2TB2MTGXPDQJIFGMW4AXJOKOE4WDKV11AROWDLEBDH
CLIENT_SECRET:QXPGNTYIPIIQZN1MYWXNKCAXWFPLTYBGI3YRJ1OYBYEGAFDZ


#### Let's explore the first neighborhood in our dataframe.
Get the neighborhood's name.

In [18]:
toronto_name_data.loc[0, 'Neighborhood']

'The Beaches'

Get the neighborhood's latitude and longitude values.

In [20]:
neighborhood_latitude = toronto_name_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_name_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_name_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.
First, let's create the GET request URL. Name your URL **url**.

In [22]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# type your answer here
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

Send the GET request

In [23]:
results = requests.get(url).json()

All the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
1,Grover Pub and Grub,Pub,43.679181,-79.297215
2,St-Denis Studios Inc.,Music Venue,43.675031,-79.288022
3,Upper Beaches,Neighborhood,43.680563,-79.292869


## 2. Explore Neighborhoods
#### Let's create a function to repeat the same process to all the neighborhoods

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe

In [27]:
toronto_name_venues = getNearbyVenues(names=toronto_name_data['Neighborhood'],
                                   latitudes=toronto_name_data['Latitude'],
                                   longitudes=toronto_name_data['Longitude']
                                  )

The Beaches
East Toronto,Riverdale
East Toronto,India Bazaar
East Toronto
Lawrence Park
Central Toronto
Central Toronto
Central Toronto
Moore Park,Central Toronto
Deer Park,Central Toronto,Rathnelly,South Hill,Central Toronto
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Downtown Toronto,Downtown Toronto
St. James Town
Berczy Park
Downtown Toronto
Downtown Toronto,Downtown Toronto,Downtown Toronto
Downtown Toronto,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Downtown Toronto
Central Toronto
Forest Hill North,Central Toronto
The Annex,Central Toronto,Yorkville
Downtown Toronto,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Downtown Toronto,Downtown Toronto,Downtown Toronto,King and Spadina,Railway Lands,South Niagara
Downtown Toronto
First Canadian Place,Underground city
Downtown Toronto
Dovercourt Village,West Toronto
Little Portugal,Trinity
West Toronto,Exhibition Place,Parkdale Village

In [28]:
print(toronto_name_venues.shape)
toronto_name_venues.head()

(1700, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,St-Denis Studios Inc.,43.675031,-79.288022,Music Venue
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"East Toronto,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


Let's check how many venues were returned for each neighborhood

In [29]:
toronto_name_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"CN Tower,Downtown Toronto,Downtown Toronto,Downtown Toronto,King and Spadina,Railway Lands,South Niagara",15,15,15,15,15,15
"Cabbagetown,St. James Town",44,44,44,44,44,44
Central Toronto,60,60,60,60,60,60
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Church and Wellesley,88,88,88,88,88,88
"Commerce Court,Downtown Toronto",100,100,100,100,100,100
"Deer Park,Central Toronto,Rathnelly,South Hill,Central Toronto",14,14,14,14,14,14
"Design Exchange,Toronto Dominion Centre",100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [30]:
print('There are {} uniques categories.'.format(len(toronto_name_venues['Venue Category'].unique())))

There are 236 uniques categories.


## 3. Analyze Each Neighborhood

In [32]:
# one hot encoding
toronto_name_onehot = pd.get_dummies(toronto_name_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_name_onehot['Neighborhood'] = toronto_name_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_name_onehot.columns[-1]] + list(toronto_name_onehot.columns[:-1])
toronto_name_onehot = toronto_name_onehot[fixed_columns]
toronto_name_onehot.shape

(1700, 236)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [33]:
toronto_name_grouped = toronto_name_onehot.groupby('Neighborhood').mean().reset_index()
toronto_name_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000
1,"CN Tower,Downtown Toronto,Downtown Toronto,Dow...",0.000000,0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Toronto,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.016667,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.01,0.010000,0.000000,0.00,0.060000,0.000000,0.030000,0.010000,0.000000
5,Church and Wellesley,0.011364,0.011364,0.011364,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.011364,0.00,0.000000,0.000000,0.00,0.000000,0.011364,0.011364,0.000000,0.011364
6,"Commerce Court,Downtown Toronto",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000
7,"Deer Park,Central Toronto,Rathnelly,South Hill...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.071429,0.000000,0.000000
8,"Design Exchange,Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.000000,0.000000,0.01,0.010000,0.000000,0.000000,0.010000,0.000000
9,"Dovercourt Village,West Toronto",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


#### Let's confirm the new size

In [35]:
toronto_name_grouped.shape

(32, 236)

#### Let's print each neighborhood along with the top 5 most common venues

In [36]:
num_top_venues = 5

for hood in toronto_name_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_name_grouped[toronto_name_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2          Restaurant  0.04
3  Seafood Restaurant  0.04
4              Bakery  0.04


----CN Tower,Downtown Toronto,Downtown Toronto,Downtown Toronto,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service  0.20
1    Airport Lounge  0.13
2  Airport Terminal  0.13
3   Harbor / Marina  0.07
4     Boat or Ferry  0.07


----Cabbagetown,St. James Town----
         venue  freq
0  Coffee Shop  0.09
1  Pizza Place  0.05
2       Market  0.05
3         Café  0.05
4          Pub  0.05


----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1  Sandwich Place  0.07
2    Dessert Shop  0.07
3  Clothing Store  0.07
4            Park  0.05


----Chinatown,Grange Park,Kensington Market----
                           venue  freq
0                           Café  0.08
1  Vegetarian / Vegan Restaurant  0.06
2                            Bar  0.05
3 

#### Let's put that into a *pandas* dataframe
First, let's write a function to sort the venues in descending order.

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_name_grouped['Neighborhood']

for ind in np.arange(toronto_name_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_name_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Farmers Market,Beer Bar,Steakhouse,Seafood Restaurant,Restaurant,Bakery
1,"CN Tower,Downtown Toronto,Downtown Toronto,Dow...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Sculpture Garden,Boutique,Plane,Boat or Ferry,Airport Gate,Airport
2,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Pub,Bakery,Market,Italian Restaurant,Pizza Place,Café,Butcher,Jewelry Store
3,Central Toronto,Sandwich Place,Coffee Shop,Clothing Store,Dessert Shop,Park,Pizza Place,Gym,Café,Italian Restaurant,Diner
4,"Chinatown,Grange Park,Kensington Market",Café,Vegetarian / Vegan Restaurant,Bar,Dumpling Restaurant,Coffee Shop,Bakery,Mexican Restaurant,Chinese Restaurant,Vietnamese Restaurant,Cocktail Bar


## 4. Cluster Neighborhoods
Run *k*-means to cluster the neighborhood into 3 clusters.

In [39]:
# set number of clusters
kclusters = 3

toronto_name_grouped_clustering = toronto_name_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_name_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [40]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_name_merged = toronto_name_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_name_merged = toronto_name_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_name_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Music Venue,Wings Joint,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
1,M4K,East Toronto,"East Toronto,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Bubble Tea Shop,Bakery,Sports Bar,Spa
2,M4L,East Toronto,"East Toronto,India Bazaar",43.668999,-79.315572,0,Park,Sandwich Place,Coffee Shop,Food & Drink Shop,Liquor Store,Brewery,Light Rail Station,Burger Joint,Burrito Place,Fast Food Restaurant
3,M4M,East Toronto,East Toronto,43.659526,-79.340923,0,Café,Coffee Shop,Park,Pizza Place,Bakery,Light Rail Station,American Restaurant,Yoga Studio,Brewery,Italian Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Bus Line,Park,Construction & Landscaping,Swim School,Wings Joint,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


Finally, let's visualize the resulting clusters

In [41]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_name_merged['Latitude'], toronto_name_merged['Longitude'], toronto_name_merged['Neighborhood'], toronto_name_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters
Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster.

#### Cluster 1

In [43]:
toronto_name_merged.loc[toronto_name_merged['Cluster Labels'] == 0, toronto_name_merged.columns[[1] + list(range(5, toronto_name_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Health Food Store,Pub,Music Venue,Wings Joint,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
1,East Toronto,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Bubble Tea Shop,Bakery,Sports Bar,Spa
2,East Toronto,0,Park,Sandwich Place,Coffee Shop,Food & Drink Shop,Liquor Store,Brewery,Light Rail Station,Burger Joint,Burrito Place,Fast Food Restaurant
3,East Toronto,0,Café,Coffee Shop,Park,Pizza Place,Bakery,Light Rail Station,American Restaurant,Yoga Studio,Brewery,Italian Restaurant
4,Central Toronto,0,Bus Line,Park,Construction & Landscaping,Swim School,Wings Joint,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
5,Central Toronto,0,Sandwich Place,Coffee Shop,Clothing Store,Dessert Shop,Park,Pizza Place,Gym,Café,Italian Restaurant,Diner
6,Central Toronto,0,Sandwich Place,Coffee Shop,Clothing Store,Dessert Shop,Park,Pizza Place,Gym,Café,Italian Restaurant,Diner
7,Central Toronto,0,Sandwich Place,Coffee Shop,Clothing Store,Dessert Shop,Park,Pizza Place,Gym,Café,Italian Restaurant,Diner
9,Central Toronto,0,Coffee Shop,Pub,Light Rail Station,Convenience Store,Sushi Restaurant,Bagel Shop,Fried Chicken Joint,Sports Bar,American Restaurant,Pizza Place
11,Downtown Toronto,0,Coffee Shop,Restaurant,Pub,Bakery,Market,Italian Restaurant,Pizza Place,Café,Butcher,Jewelry Store


#### Cluster 2

In [46]:
toronto_name_merged.loc[toronto_name_merged['Cluster Labels'] == 1, toronto_name_merged.columns[[1] + list(range(5, toronto_name_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,1,Bus Line,Trail,Jewelry Store,Sushi Restaurant,Wings Joint,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


#### Cluster 3

In [50]:
toronto_name_merged.loc[toronto_name_merged['Cluster Labels'] == 2, toronto_name_merged.columns[[1] + list(range(5, toronto_name_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,2,Playground,Trail,Tennis Court,Wings Joint,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop
10,Downtown Toronto,2,Park,Playground,Trail,Wings Joint,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
